# On the Polynomial Observer for Nonlinear Systems

<p style="font-size:16px">
This work proposes a novel <b>polynomial observable method</b> to approximate nonlinear systems with complex dynamics using a <i>data-driven decomposition approach</i>. 
The main contribution relies on providing an <b>accurate polynomial approximation</b> to devise a <b>state observer</b>, whose design may follow as usual.
Our method employs an <b>LMI condition</b> within the polynomial framework to calculate the bounding limit. <br><br>
The proposed method is evaluated in two distinct scenarios: a <b>reservoir systeml</b> with a single-state (<b>x -> liquid level</b>) system and an <b>undamped pendulum</b>  with two states (<b>x1 -> angular displacemnt</b> and <b>x2->angular velocity</b>).
</p>


<p style="font-size:16px">
This frist aplication is focused on a a reservoir system comprising two water tanks. One of them is placed at 5 m above the ground, initially empty referred to as the upper tank. Another tank at ground level is initially filled with water. A pump is activated to transfer water from the lower tank to the upper one via a pipe connection. A second pipe connects the upper and lower tanks, allowing water to flow downwards due to
gravity. The manipulated variable is the signal sent to activate the water pump, and the measured variable is the water level in the upper tank. While this setup may seem straightforward, the upper tank has a conical shape, causing its cross-sectional area to change with height, for which the output ordinary differential equation is given by the equation:
</p>
$$
\frac{dh}{dt} =  \frac{k_i \omega - k_o\sqrt{h}}{\pi r^2 + \frac{2 \pi r h (r_L - r_S)}{H} + \frac{\pi h^2 (r_L - r_S)^2}{H^2}}
$$

In [ ]:
#required libraries for code
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from SOSPy import *
from sympy import symbols, Matrix, diff
import random

import matplotlib as mpl


#tank model definitions

#tank shape
def cylinder_volume(r, R, H, h):
    return np.pi * r**2 * h + np.pi * r * h**2 * (R - r) / H + np.pi * h**3 * (R - r)**2 / (3 * H**2)

#diferential equations for tank
def ode_system(h, t, ki, ko, r, R, H, w):
    return (ki * w - ko * np.sqrt(h)) / (np.pi * r**2 + 2 * np.pi * r * h * (R - r) / H + np.pi * h**2 * (R - r)**2 / H**2)

def tank_model(xk, uk, cko):
    # Parameters
    r = 0.635
    R = 0.73
    H = 1.38
    q_nominal = 14 / 3600
    ki = q_nominal / (2 * np.pi * 60)
    ko = cko * ki * 2 * np.pi * 60 / (np.sqrt(0.60 * H))

    # Solve ODE
    t_span = [0, 1]
    ode_solution = odeint(ode_system, xk, np.linspace(t_span[0], t_span[1], 1000), args=(ki, ko, r, R, H, uk))

    # Extract the final value
    xk1 = ode_solution[-1]

    # Ensure the value is within the limits [0, H]
    xk1 = abs(xk1)
    xk1 = max(min(xk1, H), 0)

    return float(xk1), float(ki), float(ko)



: 

 <p style="font-size:16px">
     Definitions for polinomial aproximation and simulation parameters:<br><br>
     deg-> polynomial model degree <br><br>
     cko -> ko tank parameter <br><br>
     cu -> input adjust parameter <br><br>
     
 </p>

In [ ]:
deg = 7  #grau do polinomio utilizado
Ts = 1 # taxa de amostragem
Duration = 20000 # numero de amostras da simulação
cko = 0.6 
cu = 1

Ta = np.arange(0, Duration, Ts).tolist() #vetor de tempo da simulação

#print(Ta)


In [ ]:
u = 0
x = 0
xT = []
uT = []
ruido = []
for i in Ta:
    u = cu * np.pi * 60 * (1 - np.cos(2 * np.pi * i / 2500)) # Calcula a entrada
    x, _, _ = tank_model(x, u, cko) # Obetem a resposta do sistema
    eta = 1*1e-2*(random.random()-0.5)
    ruido.append(eta)
    xT.append(x)
    uT.append(u+eta)

xT = np.array(xT)
uT = np.array(uT)

In [ ]:
len(xT)
#plt.figure()
#plt.plot(xT)
#plt.plot(uT)
#plt.show()

inicialização das váriaveis para estimações polinomial e proposta

In [ ]:
xDMDp_1 = []
xDMDp = []
x1 = xT[0]
x = xT[0]


In [ ]:
ar = np.arange(0,deg+1,1)
print(ar)